In [1]:
"""
Script to delete records from sqlite database as needed.
"""

import sqlite3

In [2]:
def create_connection(db_file):
    """
    Create a database connection
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)
 
    return conn

In [ ]:
def visitid_by_date(conn, good_date):
    """
    Create a list of visitid by date
    :param conn: Connection to the database
    :param good_date: last good date in database
    :param return: list of visitids since good_date
    """
    myquery = f"SELECT visitid FROM fdinsp WHERE time_posted >=?"
    cur = conn.cursor()
    cur.execute(myquery, (good_date,))
    idlist=list(cur.fetchall())
    list_count = str(len(idlist))
            
    return idlist, list_count


In [ ]:
def delete_violations(conn, id):
    """
    Delete detailed violations by visitid
    :param conn:  Connection to the SQLite database
    :param id: visitid of the violation
    :return:
    """
    sql = "DELETE FROM violations WHERE id=?"
    cur = conn.cursor()
    cur.execute(sql, (id,))
    conn.commit()
    

In [ ]:
def delete_fdinsp(conn, good_date):
    """
    Delete summary reports since good date
    :param: Connection to database
    :param: Last good date in database
    :return:
    """
    
    sql = "DELETE FROM fdinsp WHERE time_posted >=?"
    conn = sqlite3.connect('rinspect.sqlite')
    cur = conn.cursor()
    cur.execute(sql, (good_date,))
    conn.commit() 
    

In [ ]:
def main():
    
    good_date = "2019-11-12" # Update as needed
    database = "rinspect.sqlite"
    conn = create_connection(database)
    
    # create the list for visitid
    visitids = visitid_by_date(conn, good_date)
    list_count = visitid_by_date(conn, good_date)
    visitids = [i[0] for i in visitids]
    print("There were" + list_count + "records since good date.")
    
    # use list to delete records in violations table
    for id in visitids:
        delete_violations(conn, id)
            
    # delete summary reports since good date
    delete_fdinsp(conn, good_date)
    list_count = visitid_by_date(conn, good_date)
    print("Now there are" + list_count + "records since good date.")
    
    conn.close()

In [ ]:
if __name__ == '__main__':
    main()

In [22]:
def remove_orphans(conn):
    """
    Remove records from violations table if the associated
    inspection is no longer in the fdinsp table.
    """
    
    sql = "SELECT visitid FROM violations EXCEPT SELECT visitid FROM fdinsp;"
    cur = conn.cursor()
    cur.execute(sql,)
    idlist = list(cur.fetchall())
    #list_count = str(len(idlist))
    #print(list_count)
    
    return idlist
    

In [23]:
conn = create_connection('rinspect.sqlite')
idlist = remove_orphans(conn)
visitids = [i[0] for i in idlist]

In [19]:
print(visitids)

[(5882258,), '3']
